# LivecellX tutorial: annotation by clicking single cells in the Napari UI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecellx.core import SingleCellTrajectory, SingleCellStatic
import numpy as np
import os, json, cv2, random
import cv2


In [ ]:
from livecellx.sample_data import tutorial_three_image_sys
from pathlib import Path
from livecellx.core.datasets import LiveCellImageDataset
import glob


In [ ]:
import napari
napari.__version__

## Load data

load single cells, either from masks (`prep_scs_from_mask_dataset`) or our single cells json file

Examples below: load from masks and save single cells to a json file containing single cell objects

EBSS data, 24hours

In [ ]:

# dataset_dir_path = Path(
#     "../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/XY16/"
# )

# mask_dataset_path = Path("../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/XY16/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecellx.core.io_sc import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# from livecellx.core import SingleCellStatic
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_scs_EBSS_starvation/single_cells.json",
#                                          dataset_dir="./datasets/test_scs_EBSS_starvation/datasets")


EBSS data, 5 days

In [ ]:

# dataset_dir_path = Path(
#     "../datasets/EBSS_Starvation/tifs_STAV-A549_VIM_5days_EBSS_NA_YL_Ti2e_2022-12-21001/XY3/"
# )

# mask_dataset_path = Path("../datasets/EBSS_Starvation/tifs_STAV-A549_VIM_5days_EBSS_NA_YL_Ti2e_2022-12-21001/out/XY3/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecellx.core.io_sc import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# from livecellx.core import SingleCellStatic
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_scs_EBSS_starvation/single_cells.json",
#                                          dataset_dir="./datasets/test_scs_EBSS_starvation/datasets")


In [ ]:
# from skimage.measure import regionprops
# from livecellx.core.io_sc import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_data_STAV-A549/tutorial_three_img_single_cells.json",
#                                          dataset_dir="./datasets/test_data_STAV-A549/datasets")

In [ ]:
# scs_path = "./datasets/test_data_STAV-A549/tutorial_three_img_single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/XY16/tmp_corrected_scs.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/XY1/single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/XY1/corrected_scs.json"
# single_cells = SingleCellStatic.load_single_cells_json(path=scs_path)
# dic_dataset = single_cells[0].img_dataset


load sample tutorial 3-image data

In [ ]:
from livecellx.core.io_sc import prep_scs_from_mask_dataset
# dic_dataset, mask_dataset = tutorial_three_image_sys()
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

load your own dataset...

In [ ]:
# dic_dataset = LiveCellImageDataset(...)

**Skip if you do not plan to apply CSN**   
load CSN model if you would like to use it to correct segmentation

In [ ]:
# from livecellx.core.sc_seg_operator import ScSegOperator, create_sc_seg_napari_ui
# replace the path below with your downdloaded CSN checkpoint path
# ckpt_path = r"./notebook_results/csn_models/model_v11_epoch=3282-test_loss=2.3688.ckpt"
# ScSegOperator.load_default_csn_model(path=ckpt_path, cuda=True);

## Annotating by clicking single cells in Napari

`create_scs_edit_viewer_by_interval` allows you to load the shapes from a window, (t, t+interval). You may press `n` to move forwrd the window forward, `b` to move back and `m` to load from your current step slice.
You may or may not have segmentation ready. 
### Manual segmentation and annotation
If you do not have segmentation masks, you may do annotation manually by starting with an empty list of single cell objects. All you need to do is to load an LiveCellImageDataset object.

In [ ]:
from livecellx.core.sct_operator import create_scs_edit_viewer, SctOperator, create_scs_edit_viewer_by_interval, _get_viewer_sct_operator
from livecellx.core.single_cell import create_sctc_from_scs, filter_sctc_by_time_span

In [ ]:
napari.__version__

In [ ]:
# sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset)
span_interval = 10
# viewer = create_scs_edit_viewer_by_interval(single_cells, img_dataset=dic_dataset, span_interval = 10, viewer = None)

# manual segmentation
viewer = create_scs_edit_viewer_by_interval([], img_dataset=dic_dataset, span_interval = 10, viewer = None)

# for i in range(0, len(single_cells), span_interval):
#     sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, viewer = viewer, time_span=(i, i+span_interval))
#     # set slice to the first frame of the span
#     viewer.dims.set_point(0, i)
#     # viewer.show(block=True)
#     input("Press Enter to continue...")

In [ ]:
Run all stop here!

Manually test if your annotation by clicking works and saved to the underlying single cell objects.

In [ ]:
# sct_operator = _get_viewer_sct_operator(viewer)
# test_id = 69
# sct_operator.traj_collection.get_trajectory(test_id).get_all_scs()[0].meta

In [ ]:
sct_operator = _get_viewer_sct_operator(viewer)

In [ ]:
sct_operator.traj_collection.get_all_scs()

In [ ]:
sct_operator.annotate_click_samples

In [ ]:
sct_operator.save_annotations("./tmp/test_sample")

In [ ]:
print(len(sct_operator.original_face_colors))
for key in sct_operator.shape_layer.properties.keys():
    print("property:", key + ",", "len:", len(sct_operator.shape_layer.properties[key]))
print("[sct operator] original scs len", len(sct_operator.original_scs))
print("[sct operator] actual scs len", len(sct_operator.get_all_scs()))

In [ ]:
single_cells = sct_operator.get_all_scs()
len(single_cells)

Save the corrected and annotated single cells if required

Note: now sct_operator saves single cell objects when writing samples, so you may skip the current step.

In [ ]:
SingleCellStatic.write_single_cells_json(single_cells=single_cells, path="./datasets/test_scs_EBSS_starvation/XY1/corrected_scs.json", \
                                         dataset_dir="./datasets/test_scs_EBSS_starvation/XY1/datasets")

In [ ]:
sct_operator.remove_empty_contour_sct()

How many mitosis samples did you annotate? For other labels, you may change the key below (mitosis -> label you customized)

In [ ]:
len(sct_operator.annotate_click_samples["mitosis"])

Programmatically delete an unwanted sample (e.g. the first one)

In [ ]:
# idx = 0
# label_info_key = "_annotation_label_info"
# for sc in sct_operator.annotate_click_samples["mitosis"][idx]:
#     sc.meta[label_info_key] = {}
# sct_operator.annotate_click_samples["mitosis"].pop(idx)

Save the annotations to json files.

In [ ]:
sample_out_dir = "./datasets/test_scs_EBSS_starvation/XY1/annotations"
sample_dataset_dir="./datasets/test_scs_EBSS_starvation/XY1/datasets"
res_paths = sct_operator.save_annotations(sample_out_dir=sample_out_dir, sample_dataset_dir=sample_dataset_dir)

In [ ]:
res_paths[0].as_posix()

In [ ]:
scs = sct_operator.get_all_scs()

In [ ]:
len(sct_operator.annotate_click_samples["mitosis"])

Track

In [ ]:
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)

all_scs = sct_operator.get_all_scs()

all_scs = [sc for sc in all_scs if sc.timeframe != 1]
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

Show subset of a sctc filtered by time_span

In [ ]:
from livecellx.core.sct_operator import create_scts_operator_viewer, create_scs_edit_viewer

scts_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, time_span=(145, 155))

# # If you would like to start from sctc, you can use the following code
# scts_operator = create_scts_operator_viewer(sctc, img_dataset = dic_dataset, time_span=(1, 2))

In [ ]:
# sctc.write_json(path="./EBSS_starvation_24h_xy16_annotation/single_cell_trajectory_collection.json")